In [20]:
import requests
import pandas as pd

In [21]:
# Load the movie metadata from the TSV file into a DataFrame
movie_metadata = pd.read_csv('Data/movie.metadata.tsv', sep='\t')

# Add headers to the DataFrame
movie_metadata.columns = [
    'Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 'Movie release date', 
    'Movie box office revenue', 'Movie runtime', 'Movie languages (Freebase ID:name tuples)', 
    'Movie countries (Freebase ID:name tuples)', 'Movie genres (Freebase ID:name tuples)'
]

In [22]:
import requests

# Get imdb id from the Wikipedia page ID
def get_imdb_id_from_wikipedia_page_id(page_id, language="en"):
    # Step 1: Get Wikipedia title from Page ID
    wikipedia_url = f"https://{language}.wikipedia.org/w/api.php"
    wikipedia_params = {
        "action": "query",
        "pageids": page_id,
        "format": "json"
    }
    response = requests.get(wikipedia_url, params=wikipedia_params)
    data = response.json()
    
    if "query" in data and "pages" in data["query"]:
        page = data["query"]["pages"].get(str(page_id), {})
        title = page.get("title")
        if title:
            # Step 2: Use the title to retrieve IMDb ID from Wikidata
            wikidata_url = "https://www.wikidata.org/w/api.php"
            wikidata_params = {
                "action": "wbgetentities",
                "sites": f"{language}wiki",
                "titles": title,
                "props": "claims",
                "format": "json"
            }
            wikidata_response = requests.get(wikidata_url, params=wikidata_params)
            wikidata_data = wikidata_response.json()
            
            # Extract IMDb ID (property P345)
            if "entities" in wikidata_data:
                for entity in wikidata_data["entities"].values():
                    if "P345" in entity["claims"]:  # P345 is the IMDb ID property
                        imdb_id = entity["claims"]["P345"][0]["mainsnak"]["datavalue"]["value"]
                        return imdb_id
    return None


# Get imdb id from the freebase id
def get_imdb_id_from_freebase_id_api(freebase_id):
    url = "https://www.wikidata.org/w/api.php"
    params = {
        "action": "wbgetentities",
        "format": "json",
        "props": "claims",
        "sites": "freebase",
        "titles": freebase_id
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    
    imdb_id = None
    if "entities" in data:
        for entity in data["entities"].values():
            if "P345" in entity["claims"]:  # P345 is the IMDb ID property
                imdb_id = entity["claims"]["P345"][0]["mainsnak"]["datavalue"]["value"]
    
    return imdb_id


# Apply the function to the first 10 elements of movie_metadata
# from wiki id
movie_metadata['IMDb ID from wiki'] = movie_metadata['Wikipedia movie ID'].head(10).apply(get_imdb_id_from_wikipedia_page_id)




# from freebase id
#movie_metadata['IMDb ID from freeb'] = movie_metadata['Freebase movie ID'].head(10).apply(get_imdb_id_from_freebase_id_api)



In [23]:
movie_metadata

,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages (Freebase ID:name tuples),Movie countries (Freebase ID:name tuples),Movie genres (Freebase ID:name tuples),IMDb ID from wiki
0,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",tt0245916
1,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",tt0094806
2,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",tt0094320
3,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",tt0083949
4,13696889,/m/03cfc81,The Gangsters,1913-05-29,NaN,35.0,"{""/m/06ppq"": ""Silent film"", ""/m/02h40lc"": ""Eng...","{""/m/09c7w0"": ""United States of America""}","{""/m/02hmvc"": ""Short Film"", ""/m/06ppq"": ""Silen...",tt0002894
...,...,...,...,...,...,...,...,...,...,...
81735,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",NaN
81736,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",NaN
81737,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",NaN
81738,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",NaN
